In [ ]:
from openai import OpenAI
import json
import os
import random
import tqdm

data_dir = "../../data/writing.raw_data.json"
prompts = dict(prompt=[])    
with open(data_dir,"r") as f:
    sentences = json.load(f)['original']

print(len(sentences))
print(sentences[0])

styles=["formal", "oral", "academic", "literary", "critical", "narrative", "descriptive", "lyric", "objective", "subjective", "original"] # 各种style
for _ in tqdm.tqdm(range(len(sentences))):
    style=random.choice(styles)
    prompt1 = f"Expand but not extend the paragraph in a {style} style."
    prompts['prompt'].append(prompt1)

with open(f'expand_prompt.json', 'w', encoding='utf-8') as f:
    json.dump(prompts, f, ensure_ascii=False, indent=4)

In [ ]:
from openai import OpenAI
import json
import os
import random
import tqdm

api_key = ""
client = OpenAI(api_key=api_key)
data_dir = "../../data/xsum.raw_data.json"
dataset=data_dir.split("/")[-1].split(".")[0]
model="gpt-3.5-turbo"
prompt_dir='../../data/expand_prompt.json'

result = dict(original=sentences,rewritten=[]) 

if "pubmed" in data_dir:
    questions = [i.split("Answer:")[0] for i in sentences] 
    input_sentences = [i.split("Answer: ")[1] for i in sentences]
    print(questions[0])
else:
    input_sentences=sentences


with open(data_dir,"r") as f:
    sentences = json.load(f)['original']

with open(prompt_dir,"r") as f:
    prompt = json.load(f)['prompt']
    
result = dict(original=sentences,rewritten=[])
if os.path.exists(f"../../data/expand/{model}/{dataset}_expand_{model}.raw_data.json"):
    with open(f"../../data/expand/{model}/{dataset}_expand_{model}.raw_data.json","r") as f:
        tmp = json.load(f)['rewritten']
    result = dict(original=sentences,rewritten=tmp)  
 
if "pubmed" in data_dir:
    questions = [i.split("Answer:")[0] for i in sentences] 
    input_sentences = [i.split("Answer: ")[1] for i in sentences]
    print(questions[0])
else:
    input_sentences=sentences

print(f"Start from {len(result['rewritten'])}")
for idx, sentences in tqdm.tqdm(enumerate(input_sentences[len(result['rewritten']):],start=len(result['rewritten']))):

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": f"{prompt[idx]}\n{sentences}"},
            {"role": "assistant", "content": "The expanded paragraph: "}
        ],
        temperature=0.7
    )
    if "pubmed" in data_dir:
        result['rewritten'].append(questions[idx] + "Answer: " + completion.choices[0].message.content)
    else:
        result['rewritten'].append(completion.choices[0].message.content)
    # print(completion.choices[0].message.content)

with open(f'../../data/expand/{model}/{dataset}_expand_{model}.raw_data.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)